# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [6]:
data.shape

(4870, 65)

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. What test is appropriate for this problem? Does CLT apply?

The Central Limit Theorem does not apply here since the sample size is not large enough, and the observations may not be independent of each other. 

eg. assume company A has 2 openings for a certain position and more than 2 people apply. Then the probability that someone gets this position is dependent on whether or not the candidates before them got the job or not.

The permutation method can be used to assume that there is no bias in the 'call' column based on race.

### 2. What are the null and alternate hypotheses?

H0: race has no impact on whether or not a candidate received a call. This can be formalized as checking the difference of calls received by race (no bias)
H1: race has an impact on whether or not a candidate received a call (bias exists)

### 3. Compute margin of error, confidence interval, and p-value.

In [7]:
orig_num_calls_w = data[(data['race']=='w') & (data['call']==1.0)].shape[0]
orig_num_calls_b = data[(data['race']=='b') & (data['call']==1.0)].shape[0]

orig_diff_calls = orig_num_calls_w - orig_num_calls_b
orig_diff_calls

78

In [8]:
#Use permutation method to permute the races within the dataframe 
num_trials = 1000
list_diff_calls = np.empty(num_trials)

for index in range(num_trials):
    #permute the race list
    race_list = data['race'].values
    race_list = np.random.permutation(race_list)

    #put it back into the dataframe
    data['race'] = race_list
    
    num_calls_w = data[(data['race']=='w') & (data['call']==1.0)].shape[0]
    num_calls_b = data[(data['race']=='b') & (data['call']==1.0)].shape[0]
    list_diff_calls[index] = num_calls_w - num_calls_b

In [10]:
#confidence interval
conf_interval = np.percentile(list_diff_calls, [2.5, 97.5])
p_val = np.sum(list_diff_calls >= orig_diff_calls)/1000
print('95% confidence interval for difference between the two races:', conf_interval)
print('p_val of the observed difference:', p_val)

95% confidence interval for difference between the two races: [-38.  38.]
p_val of the observed difference: 0.0


### 4. Write a story describing the statistical significance in the context or the original problem. and 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis shows that there is bias when it comes to callbacks with respect to race, but this does not show that this is the most important factor in callback success. One way to check which factors are most important to callback sucess, one can utilize a pearlson correlation test to check the correlation between each feature and the callback column.